In [1]:
import pandas as pd
# anno_1 = '../Human_anno/humananno_res/lsy_scene.xlsx'
# anno_2 = '../Human_anno/humananno_res/lsy_scene.xlsx'

# anno_1oc =pd.read_excel(anno_1, usecols=['Model','scene'])
# anno_2oc =pd.read_excel(anno_2, usecols=['Model','scene'])
# anno_1oc = anno_1oc.astype(object).where(pd.notnull(anno_1oc), None)
# anno_2oc = anno_2oc.astype(object).where(pd.notnull(anno_2oc), None)

In [2]:
import numpy as np

def rank_numbers(numbers):
    sorted_indices = sorted(range(len(numbers)), key=lambda k: numbers[k], reverse=True)
    ranks = [0] * len(numbers)

    i = 0
    while i < len(sorted_indices):
        value_indices = [i]
        while i + 1 < len(sorted_indices) and numbers[sorted_indices[i]] == numbers[sorted_indices[i + 1]]:
            i += 1
            value_indices.append(i)
        average_rank = np.mean([index + 1 for index in value_indices])
        for index in value_indices:
            ranks[sorted_indices[index]] = average_rank
        i += 1

    return ranks

def calculate_spearman_manual(values1, values2):
    n = len(values1)
    m = len(values2)
    rank1 = rank_numbers(values1)
    rank2 = rank_numbers(values2)
    d = np.array(rank1) - np.array(rank2)
    d_squared = np.square(d)
    spearman_corr = 1 - (6 * np.sum(d_squared)) / (n * (n**2 - 1))
    return spearman_corr

In [3]:
dimension = 'color'

In [4]:
import json
jsonpath = '../Human_anno/{}.json'.format(dimension)
with open(jsonpath,'r') as f:
    oc = json.load(f)

# history ="../GPT4o_eval_results/{}_gpt4eval_results.json".format(dimension)
# with open(history,'r') as f:
#     gpt4o_eval_history = json.load(f)

In [5]:
# gptvsanno1_spearman = 0
# gptvsanno2_spearman = 0
# anno1vsanno2_spearman = 0
# gptvsannomean = 0
# badeval = []

# for i in range(length):
#     if i % 3 == 0:
#         gpt4o_eval_rs = np.array(list(oc[i]['gpt4o_eval'].values()))
#         human_anno = np.array(list(oc[i]['human_anno'].values()))
#         anno1 = human_anno[:,0]
#         anno2 = human_anno[:,1]
#         annomean = (anno1 + anno2)/2
#     else:
#         gpt4o_eval_rs+= np.array(list(oc[i]['gpt4o_eval'].values()))
#         human_anno = np.array(list(oc[i]['human_anno'].values()))
#         anno1 += human_anno[:,0]
#         anno2 += human_anno[:,1]
#         annomean += (anno1 + anno2)/2

#     if i % 3 == 2:
#         gptvsanno1_spearman += calculate_spearman_manual(gpt4o_eval_rs,anno1)
#         gptvsanno2_spearman += calculate_spearman_manual(gpt4o_eval_rs,anno2)
#         anno1vsanno2_spearman += calculate_spearman_manual(anno1,anno2)
#         gptvsannomean += calculate_spearman_manual(gpt4o_eval_rs,annomean)

# gptvsanno1_spearman = 3*gptvsanno1_spearman/length
# gptvsanno2_spearman = 3*gptvsanno2_spearman/length
# anno1vsanno2_spearman = 3*  anno1vsanno2_spearman/length
# gptvsannomean = 3*gptvsannomean/length
# print("GPT vs Anno1 Spearman: ",gptvsanno1_spearman)
# print("GPT vs Anno2 Spearman: ",gptvsanno2_spearman)
# print("Anno1 vs Anno2 Spearman: ",anno1vsanno2_spearman)
# print("GPT vs AnnoMean Spearman: ",gptvsannomean)

In [6]:
models = ['cogvideox5b','gen3', 'kling','videocrafter2', 'pika', 'show1', 'lavie']
# models = ['videocrafter2', 'pika', 'show1', 'lavie']
# models = ['cogvideox5b','gen3', 'kling']
length = 54

In [7]:
gptvsanno1_spearman = np.zeros(length)
gptvsanno2_spearman = np.zeros(length)
anno1vsanno2_spearman = np.zeros(length)
gptvsannomean_spearman = np.zeros(length)
baselinevsanno1_spearman = np.zeros(length)
baselinevsanno2_spearman = np.zeros(length)
baselinevsannomean_spearman = np.zeros(length)

gptscore = np.zeros([len(models)])
anno1score = np.zeros([len(models)])
anno2score = np.zeros([len(models)])
annomeanscore =np.zeros([len(models)])
# baseline_rank = np.zeros([7])
badeval = []

for i in range(length):
    gpt4o_eval_rs = np.array(list(oc[i]['gpt4o_eval'].values()))
    human_anno = np.array(list(oc[i]['human_anno'].values()))
    # baseline_rank = np.array(list(oc[i]['baseline_rank'].values())[3:])

    anno1 = human_anno[:,0]
    # anno2 = human_anno[:,1]
    anno2 = human_anno[:,1]
    annomean = (anno1 + anno2)/2


    gptscore += gpt4o_eval_rs
    anno1score += anno1
    anno2score += anno2
    annomeanscore += annomean


    gptvsanno1 = calculate_spearman_manual(gpt4o_eval_rs,anno1)
    gptvsanno2 = calculate_spearman_manual(gpt4o_eval_rs,anno2)
    anno1vsanno2 = calculate_spearman_manual(anno1,anno2)
    gptvsannomean = calculate_spearman_manual(gpt4o_eval_rs,annomean)
    # baselinevsanno1 = calculate_spearman_manual(baseline_rank,anno1)
    # baselinevsanno2 = calculate_spearman_manual(baseline_rank,anno2)
    # baselinevsannomean = calculate_spearman_manual(baseline_rank,annomean)

    gptvsanno1_spearman[i] = gptvsanno1
    gptvsanno2_spearman[i] = gptvsanno2
    anno1vsanno2_spearman[i] = anno1vsanno2
    gptvsannomean_spearman[i] = gptvsannomean
    # baselinevsanno1_spearman[i] = baselinevsanno1
    # baselinevsanno2_spearman[i] = baselinevsanno2
    # baselinevsannomean_spearman[i] = baselinevsannomean

    if gptvsannomean <0.3:
        badeval.append(i)

gptscore = gptscore/length
anno1score = anno1score/length
anno2score = anno2score/length
annomeanscore = annomeanscore/length

print("GPT vs Anno1 Spearman: ",gptvsanno1_spearman.mean())
print("GPT vs Anno2 Spearman: ",gptvsanno2_spearman.mean())
print("Anno1 vs Anno2 Spearman: ",anno1vsanno2_spearman.mean())
print("GPT vs AnnoMean Spearman: ",gptvsannomean_spearman.mean())
# print("Baseline vs Anno1 Spearman: ",baselinevsanno1_spearman.mean())
# print("Baseline vs Anno2 Spearman: ",baselinevsanno2_spearman.mean())
# print("Baseline vs AnnoMean Spearman: ",baselinevsannomean_spearman.mean())


print("GPT average score: ",gptscore)
print("Anno1 average score: ",anno1score)
print("Anno2 average score: ",anno2score)
print("AnnoMean average score: ",annomeanscore)

GPT vs Anno1 Spearman:  0.7471891534391534
GPT vs Anno2 Spearman:  0.6800595238095235
Anno1 vs Anno2 Spearman:  0.7660383597883599
GPT vs AnnoMean Spearman:  0.7291666666666666
GPT average score:  [4.16666667 4.66666667 4.87037037 4.88888889 3.77777778 4.03703704
 3.7037037 ]
Anno1 average score:  [4.2962963  4.66666667 4.92592593 4.90740741 3.18518519 3.75925926
 3.74074074]
Anno2 average score:  [4.42592593 4.64814815 4.92592593 4.88888889 3.22222222 3.81481481
 3.90740741]
AnnoMean average score:  [4.36111111 4.65740741 4.92592593 4.89814815 3.2037037  3.78703704
 3.82407407]


In [8]:
from collections import Counter
badeval_prompt = []
for i in badeval:
    # print("spearman",gptvsannomean_spearman[i])
# print("videos",oc[i]['videos']['gen2'])
    # print("prompt",oc[i]['prompt_en'])
    badeval_prompt.append(oc[i]['prompt_en'])
    # print("GPT score",oc[i]['gpt4o_eval'])
    # print("Anno score",oc[i]['human_anno'])
    # print("GPT reasons",gpt4o_eval_history[str(i)])

In [9]:
conuter = Counter(badeval_prompt)
repeated = {k:v for k,v in conuter.items() if v>1}

In [10]:
# print(sum(repeated.values()))
# print(repeated)
# with open('D:\Astudying\VideoEval\HighlyHumanLikeVEB\Human_anno\BadEval4OverallConsistency.json','w') as f:
#     json.dump(repeated,f,indent=4)

In [11]:
gptvsanno1_spearman

array([0.83035714, 0.91964286, 1.        , 0.55357143, 0.42857143,
       1.        , 1.        , 0.80357143, 0.95535714, 0.875     ,
       0.91071429, 0.73214286, 0.47321429, 1.        , 0.89285714,
       0.63392857, 0.41071429, 0.91964286, 0.5625    , 0.90178571,
       0.54464286, 0.92857143, 0.15178571, 0.78571429, 0.78571429,
       0.76785714, 0.85714286, 0.54464286, 1.        , 0.38392857,
       0.67857143, 0.78571429, 0.64285714, 0.86607143, 0.61607143,
       0.89285714, 1.        , 0.97321429, 0.8125    , 0.89285714,
       0.65178571, 0.67857143, 0.78571429, 1.        , 0.19642857,
       0.86607143, 0.86607143, 0.86607143, 0.67857143, 0.22321429,
       0.65178571, 0.41071429, 0.9375    , 0.82142857])

In [12]:
anno_1oc

NameError: name 'anno_1oc' is not defined